In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import warnings

warnings.filterwarnings('ignore')
%matplotlib inline

In [7]:
data = pd.read_excel('Online retail.xlsx', header= None)

In [73]:
data.shape

(7501, 1)

In [8]:
data.head()

,0
0,"shrimp,almonds,avocado,vegetables mix,green gr..."
1,"burgers,meatballs,eggs"
2,chutney
3,"turkey,avocado"
4,"mineral water,milk,energy bar,whole wheat rice..."


### Preprocessing the data

In [12]:
transactions = data[0].apply(lambda x: x.split(',')).tolist()

In [13]:
transactions

[['shrimp',
  'almonds',
  'avocado',
  'vegetables mix',
  'green grapes',
  'whole weat flour',
  'yams',
  'cottage cheese',
  'energy drink',
  'tomato juice',
  'low fat yogurt',
  'green tea',
  'honey',
  'salad',
  'mineral water',
  'salmon',
  'antioxydant juice',
  'frozen smoothie',
  'spinach',
  'olive oil'],
 ['burgers', 'meatballs', 'eggs'],
 ['chutney'],
 ['turkey', 'avocado'],
 ['mineral water', 'milk', 'energy bar', 'whole wheat rice', 'green tea'],
 ['low fat yogurt'],
 ['whole wheat pasta', 'french fries'],
 ['soup', 'light cream', 'shallot'],
 ['frozen vegetables', 'spaghetti', 'green tea'],
 ['french fries'],
 ['eggs', 'pet food'],
 ['cookies'],
 ['turkey', 'burgers', 'mineral water', 'eggs', 'cooking oil'],
 ['spaghetti', 'champagne', 'cookies'],
 ['mineral water', 'salmon'],
 ['mineral water'],
 ['shrimp',
  'chocolate',
  'chicken',
  'honey',
  'oil',
  'cooking oil',
  'low fat yogurt'],
 ['turkey', 'eggs'],
 ['turkey',
  'fresh tuna',
  'tomatoes',
  'spagh

In [16]:
from mlxtend.preprocessing import TransactionEncoder

In [17]:
def preprocess_data(transactions):
    """
    Converts the raw data into a format suitable for association rule mining.
    Args:
    - df (DataFrame): The pandas DataFrame containing the data.
    
    Returns:
    - transaction_df (DataFrame): The one-hot encoded DataFrame for transactions.
    """
    te = TransactionEncoder()
    te_ary = te.fit(transactions).transform(transactions)
    transaction_df = pd.DataFrame(te_ary, columns=te.columns_)
    return transaction_df

In [18]:
transaction_df = preprocess_data(transactions= transactions)

In [20]:
transaction_df.head()

,asparagus,almonds,antioxydant juice,asparagus,avocado,babies food,bacon,barbecue sauce,black tea,blueberries,...,turkey,vegetables mix,water spray,white wine,whole weat flour,whole wheat pasta,whole wheat rice,yams,yogurt cake,zucchini
0,False,True,True,False,True,False,False,False,False,False,...,False,True,False,False,True,False,False,True,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,False


### Frequent item sets

In [21]:
from mlxtend.frequent_patterns import apriori, association_rules

In [28]:
def apply_apriori(transaction_df, min_support=0.01):
    """
    Applies the Apriori algorithm to find frequent itemsets.
    Args:
    - transaction_df (DataFrame): The one-hot encoded DataFrame for transactions.
    - min_support (float): Minimum support threshold.
    
    Returns:
    - frequent_itemsets (DataFrame): The frequent itemsets found by the Apriori algorithm.
    """
    frequent_itemsets = apriori(transaction_df, min_support=min_support, use_colnames=True)
    return frequent_itemsets


In [29]:
frequent_itemsets = apply_apriori(transaction_df= transaction_df)

In [30]:
frequent_itemsets

,support,itemsets
0,0.020397,(almonds)
1,0.033329,(avocado)
2,0.010799,(barbecue sauce)
3,0.014265,(black tea)
4,0.011465,(body spray)
...,...,...
252,0.011065,"(ground beef, milk, mineral water)"
253,0.017064,"(ground beef, spaghetti, mineral water)"
254,0.015731,"(spaghetti, milk, mineral water)"
255,0.010265,"(spaghetti, olive oil, mineral water)"


### Association Rules

In [80]:
def generate_association_rules(frequent_itemsets, min_support=0.01, min_confidence=0.4, min_lift=1.0):
    """
    Generates association rules from frequent itemsets with specified thresholds for support, confidence, and lift.
    Args:
    - frequent_itemsets (DataFrame): The frequent itemsets.
    - min_support (float): Minimum support threshold.
    - min_confidence (float): Minimum confidence threshold.
    - min_lift (float): Minimum lift threshold.
    
    Returns:
    - rules (DataFrame): The association rules generated from the frequent itemsets.
    """
    rules = association_rules(frequent_itemsets, metric="lift", min_threshold=min_lift)
    rules = rules[(rules['support'] >= min_support) & (rules['confidence'] >= min_confidence)]
    return rules


In [81]:
rules = generate_association_rules(frequent_itemsets= frequent_itemsets)

In [82]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
218,(ground beef),(mineral water),0.098254,0.238368,0.040928,0.416554,1.747522,1.0,0.017507,1.305401,0.474369,0.138413,0.233952,0.294127
262,(olive oil),(mineral water),0.065858,0.238368,0.027596,0.419028,1.757904,1.0,0.011898,1.310962,0.461536,0.099759,0.237201,0.267400
268,(salmon),(mineral water),0.042528,0.238368,0.017064,0.401254,1.683336,1.0,0.006927,1.272045,0.423972,0.064679,0.213864,0.236421
272,(soup),(mineral water),0.050527,0.238368,0.023064,0.456464,1.914955,1.0,0.011020,1.401255,0.503221,0.086760,0.286354,0.276610
306,"(chocolate, eggs)",(mineral water),0.033196,0.238368,0.013465,0.405622,1.701663,1.0,0.005552,1.281394,0.426498,0.052169,0.219600,0.231055
318,"(ground beef, chocolate)",(mineral water),0.023064,0.238368,0.010932,0.473988,1.988472,1.0,0.005434,1.447937,0.508837,0.043640,0.309362,0.259925
324,"(chocolate, milk)",(mineral water),0.032129,0.238368,0.013998,0.435685,1.827780,1.0,0.006340,1.349656,0.467922,0.054574,0.259071,0.247205
336,"(chocolate, spaghetti)",(mineral water),0.039195,0.238368,0.015865,0.404762,1.698053,1.0,0.006522,1.279541,0.427860,0.060621,0.218470,0.235658
342,"(ground beef, eggs)",(mineral water),0.019997,0.238368,0.010132,0.506667,2.125563,1.0,0.005365,1.543848,0.540342,0.040816,0.352268,0.274586
348,"(eggs, milk)",(mineral water),0.030796,0.238368,0.013065,0.424242,1.779778,1.0,0.005724,1.322834,0.452053,0.051015,0.244047,0.239526


# Analysis of Association Rules: Customer Purchasing Behavior

## Objective
The goal of this analysis is to explore the association rules derived from a large dataset of customer transactions. By analyzing these rules, we aim to uncover meaningful relationships between products that can provide valuable insights into customer purchasing behavior.

## Overview of Generated Rules
The dataset contains association rules generated using the **Apriori algorithm**. These rules are evaluated based on several key metrics, including:

- **Support**: The proportion of transactions that contain both the antecedent and consequent items.
- **Confidence**: The likelihood that the consequent is purchased when the antecedent is purchased.
- **Lift**: The ratio of the observed support to the expected support if the antecedent and consequent were independent.
- **Leverage**: The difference between the observed support and the expected support.
- **Conviction**: A measure of how often the consequent is bought when the antecedent is bought, adjusted for the antecedent's frequency.
- **Zhang's Metric**: A measure of the statistical dependence between the antecedent and consequent.
- **Jaccard**: A measure of similarity between the antecedent and consequent.
- **Certainty Factor**: A measure of the certainty of the rule.
- **Kulczynski**: A measure of the association strength between the antecedent and consequent.

The following are the most notable findings from the analysis:

## Key Insights from the Association Rules

### 1. **Frequent Co-occurrences**
- **Ground Beef and Mineral Water**:
  - **Support**: 4.09%
  - **Confidence**: 41.66%
  - **Lift**: 1.75
  
  This rule indicates that when customers purchase **ground beef**, they are highly likely to purchase **mineral water** as well. The high **confidence** value suggests that there is a strong association between these two items. The **lift** value greater than 1 further confirms that the association is not random but rather indicates a meaningful connection.

- **Olive Oil and Mineral Water**:
  - **Support**: 2.76%
  - **Confidence**: 41.90%
  - **Lift**: 1.76
  
  Customers who buy **olive oil** are also highly likely to purchase **mineral water**, with a slightly lower support but still a strong relationship based on **confidence** and **lift** values.

### 2. **Multiple Item Combinations**
- **Chocolate, Eggs, and Mineral Water**:
  - **Support**: 1.35%
  - **Confidence**: 40.56%
  - **Lift**: 1.70
  
  This rule highlights that the combination of **chocolate** and **eggs** is often followed by the purchase of **mineral water**. Although the support is relatively low, the **confidence** suggests a solid correlation between these items, making it interesting for cross-promotions.

- **Ground Beef, Eggs, and Mineral Water**:
  - **Support**: 1.01%
  - **Confidence**: 47.40%
  - **Lift**: 1.99
  
  The combination of **ground beef** and **eggs** frequently leads to the purchase of **mineral water**. This is an interesting combination that can be explored for bundling promotions or discounts, as customers purchasing **ground beef** and **eggs** show a high likelihood of purchasing **mineral water** as well.

### 3. **Product Pairings for Marketing Opportunities**
- **Milk and Frozen Vegetables**:
  - **Support**: 1.11%
  - **Confidence**: 46.89%
  - **Lift**: 1.97
  
  Customers who buy **milk** are likely to also purchase **frozen vegetables**. This pattern can be utilized for product pairings in marketing strategies, where promotions for **milk** can be bundled with **frozen vegetables**.

- **Milk and Spaghetti**:
  - **Support**: 1.57%
  - **Confidence**: 44.36%
  - **Lift**: 1.86
  
  **Milk** and **spaghetti** frequently co-occur in transactions, suggesting a potential opportunity for cross-selling or bundled promotions targeting customers who purchase **milk**.

### 4. **Low Support, High Confidence Relationships**
- **Chocolate, Milk, and Mineral Water**:
  - **Support**: 1.40%
  - **Confidence**: 43.57%
  - **Lift**: 1.83
  
  While the support for this rule is low, the **confidence** and **lift** values indicate that when customers purchase **chocolate** and **milk**, they are likely to also purchase **mineral water**. This might be valuable for targeted marketing campaigns.

## Recommendations for Business Strategy

### 1. **Product Bundling**
The identified associations suggest several potential bundling opportunities for products like **mineral water**, **ground beef**, **olive oil**, **eggs**, and **spaghetti**. Marketers can create attractive bundles around these items to encourage higher customer spend.

### 2. **Cross-Promotions**
By leveraging rules such as **milk and frozen vegetables** and **chocolate, eggs and mineral water**, businesses can design **cross-promotion strategies** that target customers who purchase certain items together, offering discounts or promotions for complementary products.

### 3. **Personalized Recommendations**
The discovered rules could be used for **personalized recommendation systems** on e-commerce platforms, where the system suggests items that customers are likely to purchase together based on the items in their shopping cart.

### 4. **Targeted Advertising**
For products like **ground beef** and **mineral water**, **olive oil** and **mineral water**, or **chocolate** and **eggs**, targeted advertising can be designed to feature these products together, based on the strong association found in the rules.

## Conclusion
The analysis of the association rules provides valuable insights into customer purchasing behavior. By identifying frequent product pairings and co-occurrences, businesses can make informed decisions regarding product placements, marketing strategies, and promotional offers. Understanding these relationships helps businesses drive sales by strategically aligning their product offerings with customer preferences.